# NDVI fourier -> ag

Sources:
- https://www.scielo.br/pdf/pab/v47n9/12.pdf

In [1]:
from pathlib import Path
import pickle

import numpy as np
import rasterio as rio
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [2]:
dir_in = Path("../data/ndvi-weekly/")
dir_out = dir_in / "../ndvi-proc/"
tifs = [f for f in dir_in.iterdir() if f.suffix == ".tif"]

In [3]:
prof = None
with rio.open(tifs[0]) as ds:
    prof = ds.profile

In [ ]:
arrs = []
for f in tifs:
    with rio.open(f) as ds:
        arrs.append(ds.read(1))

In [ ]:
st = np.stack(arrs)
st.shape

In [ ]:
with open("stack.pickle", "wb") as f:
    pickle.dump(st, f)

In [ ]:
with open("stack.pickle", "rb") as f:
    st = pickle.load(f)

In [ ]:
# use abs to convert to real numbers
st = np.abs(np.fft.fft(st, axis=0))
st.shape

In [ ]:
with open("fft.pickle", "wb") as f:
    pickle.dump(st, f)

In [4]:
with open("fft.pickle", "rb") as f:
    st = pickle.load(f)

In [6]:
st.shape, st.dtype

((52, 7127, 5567), dtype('float32'))

In [15]:
bands = 6
ft = st[:bands, :, :]
ft.shape

(6, 7127, 5567)

In [16]:
prof.update(count=bands)
with rio.open(dir_out / "weekly-fourier-6-morebands.tif", "w", **prof) as ds:
    ds.write(ft.astype(np.float32))

# Now let's do ML
- https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans

## KMeans
How many clusters?

In [17]:
ft2 = np.moveaxis(ft, 0, -1)
# ft2 = ft2[:, :, 1:]
ft2.shape

(7127, 5567, 6)

In [18]:
X = ft2.reshape((-1, bands))
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
X.shape

(39676009, 6)

In [19]:
n_clusters = 5
y = KMeans(n_clusters=n_clusters).fit_predict(X)
y.shape

(39676009,)

In [20]:
y_out = y.reshape(st.shape[1:])
y_out.shape

(7127, 5567)

In [ ]:
plt.imshow(y_out)

In [21]:
prof.update(count=1)
with rio.open(dir_out / "weekly-pred-2-morebands.tif", "w", **prof) as ds:
    ds.write(y_out.astype(np.float32), indexes=1)